In [13]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!ls "/content/drive/My Drive/Colab Notebooks/"

'image recognition.ipynb'  'Model With Greater Dataset.ipynb'   train1.zip
 ImageRec.zip		   'Optimization of Layers'	        TraingData.zip
 Model1.ipynb		    Supernova.ipynb		        train.zip


In [15]:
train_file = "/content/drive/My Drive/Colab Notebooks/TraingData.zip"

In [16]:
import zipfile

with zipfile.ZipFile(train_file, 'r') as z:
    z.extractall()

In [17]:
import os
import cv2

def load_image(file_path):
    return cv2.imread(file_path)

def extract_label(file_name):
    return 1 if "Supernova" in file_name else 0

train_path = "./TraingData/"
image_files = os.listdir(train_path)
train_images = [load_image(train_path + file) for file in image_files]
train_labels = [extract_label(file) for file in image_files]

In [18]:
def preprocess_image(img, side=96):
    min_side = min(img.shape[0], img.shape[1])
    img = img[:min_side, :min_side]
    img = cv2.resize(img, (side,side))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img / 255.0

In [19]:
for i in range(len(train_images)):
    train_images[i] = preprocess_image(train_images[i])

In [20]:
import numpy as np

train_images = np.expand_dims(train_images, axis=-1)
train_labels = np.array(train_labels)
print(train_images.shape, train_labels.shape)

(306, 96, 96, 1) (306,)


In [30]:
import tensorflow as tf
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu", input_shape=(96,96,1)))
model.add(tf.keras.layers.MaxPool2D())

model.add(tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D())

model.add(tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128,activation="relu"))
model.add(tf.keras.layers.Dense(2, activation="softmax"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 96, 96, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)       

In [31]:
opt = tf.keras.optimizers.Adam(lr=0.000001)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [33]:
model.fit(train_images, train_labels, epochs=10, batch_size=1)

Epoch 1/10
306/306 [==============================] - 7s 22ms/step - loss: 0.6877 - accuracy: 0.5882
Epoch 2/10
306/306 [==============================] - 7s 22ms/step - loss: 0.6821 - accuracy: 0.5915
Epoch 3/10
306/306 [==============================] - 7s 23ms/step - loss: 0.6795 - accuracy: 0.5980
Epoch 4/10
306/306 [==============================] - 7s 23ms/step - loss: 0.6754 - accuracy: 0.5850
Epoch 5/10
306/306 [==============================] - 7s 23ms/step - loss: 0.6715 - accuracy: 0.5850
Epoch 6/10
306/306 [==============================] - 7s 23ms/step - loss: 0.6694 - accuracy: 0.5882
Epoch 7/10
306/306 [==============================] - 7s 22ms/step - loss: 0.6668 - accuracy: 0.5882
Epoch 8/10
306/306 [==============================] - 7s 22ms/step - loss: 0.6679 - accuracy: 0.5882
Epoch 9/10
306/306 [==============================] - 7s 22ms/step - loss: 0.6637 - accuracy: 0.5882
Epoch 10/10
306/306 [==============================] - 7s 22ms/step - loss: 0.6655 - accura